# HOTRG with adaptive bond dimension $\chi$

In [1]:
import numpy as np
from numpy.typing import ArrayLike
import hotrg
import ipywidgets as widgets
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ["svg"]
import itertools
from tqdm.auto import tqdm
import cProfile
import opt_einsum as oe

In [2]:
temperature = widgets.FloatSlider(min=2.0, max=5.0)
temperature

FloatSlider(value=2.0, max=5.0, min=2.0)

In [5]:
chi = widgets.IntRangeSlider(min=8, max=16, value=(8, 12))
chi

IntRangeSlider(value=(8, 12), max=16, min=8)

In [11]:
tol = widgets.FloatLogSlider(value=1e-6, base=10, min=-15, max=-4, step=1)
tol

FloatLogSlider(value=1e-06, max=-4.0, min=-15.0, step=1.0)

In [23]:
def run_iterations(H: ArrayLike, T: ArrayLike, magnetization: list[float], tol: float = 1e-6, max_bond: int = 8, max_it: int = 20):
	stop = False
	n = 0

	while not stop:
		for permutator in itertools.islice(itertools.cycle([(0,1,3,2), (2,3,0,1)]), 4):
			H, T = hotrg.iterate(H, T, max_bond)
			H = H.transpose(permutator)
			T = T.transpose(permutator)

		magnetization.append(hotrg.trace(H)/hotrg.trace(T))
		ratio = magnetization[-2] / magnetization[-1]

		print(f"magnetization = {magnetization[-1]} ({ratio=})")

		stop = 1 - tol < ratio < 1 + tol
		n += 1

	return H, T, n

In [30]:
T = hotrg.ising.partition_tensor(temperature.value)
H = hotrg.ising.magnetization(temperature.value)
magnetization = [hotrg.trace(H)/hotrg.trace(T)]

max_bond = chi.value[0]

tolerance = tol.value
stop = False

with cProfile.Profile() as profile:
	for max_bond in range(chi.value[0], chi.value[1]+1, 2):
		H, T, n = run_iterations(H, T, magnetization, max_bond=max_bond, tol=tolerance)
		print(f"----\n{max_bond=} completed in {n=} iterations with {tolerance=}\n")
		# tolerance /= 10

magnetization = -2.296598941053157e-16 (ratio=-0.0)
magnetization = -2.8855768381777558e-15 (ratio=0.07958890266472547)
magnetization = -1.938728847367614e-11 (ratio=0.00014883859814103257)
magnetization = 1.9152665325075858e-07 (ratio=-0.00010122501565509578)
magnetization = 2.614339839897887e-06 (ratio=0.07326004459245795)
magnetization = 4.2111705087438905e-05 (ratio=0.062081073052482345)
magnetization = 0.0006748383661961874 (ratio=0.062402654023372896)
magnetization = 0.01079978863424202 (ratio=0.062486256819557724)
magnetization = 0.1707678169652989 (ratio=0.06324252910275596)
magnetization = 0.9075056352842724 (ratio=0.18817273449966687)
magnetization = 0.9117301360466028 (ratio=0.9953665009027249)
magnetization = 0.9117299721453299 (ratio=1.0000001797695348)
magnetization = 0.9117298082873208 (ratio=1.0000001797221147)
magnetization = 0.9117296443593851 (ratio=1.0000001797988436)
magnetization = 0.911729480661665 (ratio=1.0000001795463715)
magnetization = 0.9117293167098551 (ra

In [14]:
profile.print_stats("tottime")

         68644 function calls (67972 primitive calls) in 14.511 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       96   12.079    0.126   12.141    0.126 linalg.py:1477(svd)
 1128/840    1.608    0.001   13.855    0.016 {built-in method numpy.core._multiarray_umath.implement_array_function}
     1056    0.138    0.000    0.138    0.000 {method 'reshape' of 'numpy.ndarray' objects}
       48    0.121    0.003    0.142    0.003 tn.py:19(error_right)
       48    0.119    0.002    0.132    0.003 tn.py:15(error_left)
       48    0.109    0.002   12.278    0.256 hosvd.py:43(hosvd_sides)
      720    0.065    0.000    0.065    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      288    0.058    0.000    0.058    0.000 {method 'astype' of 'numpy.ndarray' objects}
      216    0.022    0.000    0.110    0.001 contract.py:93(contract_path)
      288    0.015    0.000    1.656    0.006 numeric.py:949(tensordot)
       96    0.